In [1]:
#Spark is auto put data in RDD to distribute to the cluster

#RDD is a immutable distributed system

In [ ]:
conf = SparkConf().setMaster("local").setAppName("My App") #local ==> to local 執行續
sc = SparkContext(conf = conf)

In [ ]:
#1. 載入外部資料 2. 在驅動程式中分派一個物件
sc.stop()

### 1.

In [ ]:
from pyspark import SparkConf, SparkContext

inputRDD = sc.textFile("/home/jordan/Desktop/input.txt")

### 2.

In [21]:
lines = sc.parallelize(["pandas","i like pandas"])

In [10]:
# if build a RDD then have two thing can use:

In [11]:
pythonlines = lines.filter(lambda line: "INFO" in line)

### Action()

In [12]:
# in calucate of RDD ,it will return the 驅動程式 or save to outside save machine ex:HDFS
pythonlines.first()

u'INFO This is a message with content'

In [13]:
 #RDD is a lazy fashion only when they be action RDD will be calucate
#預設 每次對RDD 執行一個行動操作,預設是要重新計算結果。if 要在多個行動中重複使用一個RDD,RDD.persist() RDD 將會放在記憶體中

In [14]:
pythonlines.persist()

PythonRDD[5] at RDD at PythonRDD.scala:48

In [15]:
pythonlines.count()

4

In [16]:
pythonlines.first()

u'INFO This is a message with content'

# 流程

In [ ]:
1. build RDD from outside data
2. get new RDD from transcation actions. example:filter()
3. save RDD by persist()
4. 啟動平行化計算 by action like count() and first()

In [36]:
# Transformations 
warningsRDD = inputRDD.filter(lambda x: "WARN" in x)
errorsRDD = inputRDD.filter(lambda x: "ERROR" in x)

In [37]:
#union warn and errors
badLinesRDD = errorsRDD.union(warningsRDD)

In [38]:
badLinesRDD

UnionRDD[4] at union at NativeMethodAccessorImpl.java:0

In [42]:
# Actions
print "Input had " + str(badLinesRDD.count()) + " concerning lines"
print "Here are 10 examples:"
print("------")
for line in badLinesRDD.take(2):
    print line

Input had 3 concerning lines
Here are 10 examples:
------
ERROR Something bad happened
WARN This is a warning


In [41]:
for line in badLinesRDD.collect(): #collect() get the one whole RDD, need to be inside the computer,is not useful to put in a big data
    print line

ERROR Something bad happened
WARN This is a warning
WARN More details on the bad thing


In [43]:
### 大資料不適合用collect(),會被寫到一個分散式儲存系統 example: saveAsTextFile() 和 saveAsSequenceFile()

### 慵懶評估

### pass func to Spark

In [ ]:
1. word = rdd.filter(lambda s: "error" in s)

2. def containsError(s):
    return "error" in s
3. word = rdd.filter(contains)

### 傳遞函式時會序列化包含該函式的物件,spark會傳送整個物件到工作節點,而你只需要整個物件的某一小部分而已

In [ ]:
class SearchFunctions(object):
    def __init__(self, query):
        self.query = query
    def isMatch(self, s):
        return self.query in s
    def getMatchesFunctionReference(self, rdd):
        #problem is "self.isMatch" is base on all "self"
        return rdd.filter(self.isMatch)
    

In [44]:
### correct one is 
class SearchFunctions(object):
    ''''''
    def getMatchesFunctionReference(self, rdd):
        query = self.query
        return rdd.filter(lambda x: query in x)

In [45]:
### map() 會取得一個函式,並將其應用於RDD中的每一個元素 > 回傳型態不需與輸入型態相同 ==> 許多串列的RDD
nums = sc.parallelize([2, 3, 4, 5, 6])
squared = nums.map(lambda x: x * x).collect()
for num in squared:
    print "%i " %(num)

4 
9 
16 
25 
36 


In [46]:
squared

[4, 9, 16, 25, 36]

In [47]:
### flatMap() 作用於RDD每個元素上, 回傳一個內含所有回傳值的迭代器 ==> 所有串列的元素集合
lines = sc.parallelize(["hello world", "hi"])
words = lines.flatMap(lambda line: line.split(" "))
words.collect()

['hello', 'world', 'hi']

In [ ]:

flatMap()and map() differ 
rdd = {"coffee panda","happy panda"}
maprdd = {["coffee","panda"],["happy","panda"]}
flatrdd = {"coffee","panda","happy","panda"}

## 以下RDD 需相同型別

### union(RDD) (if 有資料副本,結果也會包含副本資料需要透過distinct()修正)

### intersection(RDD) (效率會差很多,必須跨網路進行洗牌,以變識副本)

### subtract(RDD)

### cartesian(RDD) (對大型RDD操作非常昂貴)

In [48]:
intersection = errorsRDD.intersection(warningsRDD)
intersection

PythonRDD[25] at RDD at PythonRDD.scala:48

In [49]:
intersection.collect()

[]

In [51]:
inputRDD.intersection(warningsRDD).collect()

[u'WARN More details on the bad thing', u'WARN This is a warning']

In [52]:
inputRDD.subtract(warningsRDD).collect()

[u'INFO This is a message with content',
 u'ERROR Something bad happened',
 u'INFO Here are more messages',
 u'(empty line)',
 u'(empty line)',
 u'INFO back to normal messages',
 u'INFO This is a some other content']

In [54]:
errorsRDD.cartesian(warningsRDD).collect()

[(u'ERROR Something bad happened', u'WARN This is a warning'),
 (u'ERROR Something bad happened', u'WARN More details on the bad thing')]

### reduce()

In [55]:

rdd = sc.parallelize([2, 3, 4, 5, 6])

In [61]:
y = 1
rdd.reduce(lambda x, y:x+ y)

20

In [62]:
print "sum is ",rdd.reduce(lambda x, y:x+ y)

sum is  20


In [64]:
### aggregate() 免於回傳相同型別RDD限制 to return a value of pair

In [66]:
sumCount = rdd.aggregate((0, 0),
                        (lambda acc, value: (acc[0] + value, acc[1] + 1)),
                        (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))


In [70]:
sumCount[0]/float(sumCount[1])

4.0

### take(n) => return n element,try te read 最少分割,可能不會按照預期順序回傳元素

### top() => return 預期順序回傳元素

### foreach() => 對RDD中的每個元素執行行動操作,但不回傳任何值


### persist() 在python中 當寫入到硬碟和非heap 的記憶體區段 這些資料是被序列化的

In [71]:
sc.stop()